# Analyse Multidimensionnelle des Chansons de Rap : Complexité, Politique et Vulgarité

L'analyse présentée dans ce notebook se concentre sur les chansons de rap issues du top 50 français, en explorant leur complexité lexicale, leur message politique et leur niveau de vulgarité. Les données utilisées proviennent d'un fichier Parquet disponible sur Hugging Face, à l'adresse https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet, qui contient des informations détaillées sur les chansons, notamment leur titre, artiste, année de sortie, complexité lexicale, thèmes principaux et tonalité émotionnelle.

L'objectif de cette analyse est de créer un diagramme à trois axes pour visualiser les chansons de rap en fonction de l'artiste, de leur complexité lexicale, de leur message politique et de leur niveau de vulgarité. Pour cela, nous allons agréger les valeurs textuelles en valeurs numériques et calculer la moyenne par artiste, afin de comparer les différents artistes de rap sur ces critères.

## Méthodologie

La méthodologie utilisée dans cette analyse consiste en plusieurs étapes. Tout d'abord, nous avons sélectionné les données relatives aux chansons de rap et avons transformé les niveaux textuels de complexité lexicale et de message politique en valeurs numériques. Ensuite, nous avons calculé la moyenne de ces valeurs, ainsi que de la vulgarité, pour chaque artiste. Les données ont été regroupées par artiste et triées pour faciliter la visualisation. Nous avons utilisé la bibliothèque Plotly pour créer un diagramme à trois axes interactif, permettant de visualiser les relations entre la complexité lexicale, le message politique et la vulgarité pour les différents artistes de rap.

Cette démarche permet d'obtenir une vue d'ensemble de la manière dont les artistes de rap abordent les thèmes complexes, les messages politiques et la vulgarité dans leurs chansons, et de comparer leurs approches respectives de manière visuelle et interactive.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH 
  -- Transformation des niveaux textuels en valeurs numériques
  artiste_complexite AS (
    SELECT 
      "artiste",
      CASE "complexite_lexicale"
        WHEN 'faible' THEN 1
        WHEN 'moyen' THEN 2
        WHEN 'élevé' THEN 3
        ELSE NULL
      END AS complexite_lexicale_num,
      CASE "message_politique"
        WHEN 'faible' THEN 1
        WHEN 'moyen' THEN 2
        WHEN 'élevé' THEN 3
        ELSE NULL
      END AS message_politique_num,
      "vulgarite"
    FROM loaded_dataset
    WHERE "style_musique" = 'rap'
  )

SELECT 
  "artiste",
  AVG("complexite_lexicale_num") AS complexite_moyenne,
  AVG("message_politique_num") AS message_politique_moyen,
  AVG("vulgarite") AS vulgarite_moyenne
FROM artiste_complexite
GROUP BY "artiste"
ORDER BY "artiste" """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée pour cette visualisation de données est Plotly Express, qui permet de créer des graphiques interactifs en 3D. Cette technologie est adaptée pour représenter des données multidimensionnelles complexes de manière claire et engageante. Elle offre une grande flexibilité pour personnaliser l'apparence et le comportement de la visualisation.

In [3]:
import pandas as pd
import duckdb as ddb
import plotly.express as px

dataviz = px.scatter_3d(
    df.sort_values('vulgarite_moyenne', ascending=False),
    x='complexite_moyenne',
    y='message_politique_moyen',
    z='vulgarite_moyenne',
    color='vulgarite_moyenne',
    color_continuous_scale='Reds',
    hover_name='artiste',
    size_max=30,
    opacity=0.8,
    labels={
        'complexite_moyenne': 'Complexité lexicale',
        'message_politique_moyen': 'Message politique',
        'vulgarite_moyenne': 'Vulgarité'
    }
)

dataviz.update_layout(
    title='Analyse comparative des artistes de rap',
    scene=dict(
        xaxis=dict(title='Complexité lexicale', dtick=0.5),
        yaxis=dict(title='Message politique', dtick=0.5),
        zaxis=dict(title='Vulgarité', dtick=1)
    ),
    margin=dict(l=20, r=20, t=40, b=20),
    font=dict(family="Arial", size=12),
    coloraxis_colorbar_title_text='Vulgarité'
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*